This is v2 of the control law propagator.

The major update to the CLP is that there is an extra loop for propagation; Inside a single bucket, use a fixed step size and hold u constant while we propagate q and p.  Then, using those values for p and q, propagate u.

There will be three main modules:

- 1) Propagator

- 2) sliding window

- 3) window filter

In [1]:
import numpy as np
import scipy as sp
import ode
# print t_terminal
from sliding_window import *
# print t_terminal

In [2]:
# Inputs:
# q_0 = np.array([1])
# p_0 = np.array([0])
# u_0 = np.array([0])
# qpu_vec = np.concatenate([q_0, p_0, u_0])

# t_0 = 0
# T =  2
# K=10

# n_s = 10
# integrateTol = 10**-3
# integrateMaxIter = 40
# state_dim = 1
# Gamma = 1

# grab sliding window from module sliding_window.py
mySlidingWindow = SlidingWindowExample()

In [3]:
print mySlidingWindow.t_0, mySlidingWindow.T, mySlidingWindow.t_terminal, mySlidingWindow.K

0 2 2 1


In [4]:
# (T-t_0)/float(2*K)

In [5]:
def H_T_p(q,p,u):
    # for q-dot
    return np.ones(np.shape(q))*0

def H_T_q(q,p,u):
    # for p-dot
    return np.ones(np.shape(p))*0
    
def Q_u(q,p,u):
    # for u-dot
    return np.ones(np.shape(u))*0
    

In [6]:
# def rhs(t, qpu_vec, **kwargs):
#     # TODO: make sure that the functions that this calls are available
#     state_dim = kwargs['state_dim']
#     Gamma = kwargs['Gamma']
#     q = qpu_vec[:state_dim]
#     p = qpu_vec[state_dim:2*state_dim]
#     u = qpu_vec[2*state_dim:]
#     q_dot =  H_T_p(q,p,u)
#     p_dot = -1*H_T_q(q,p,u)
#     u_dot = -Gamma*Q_u(q,p,u)
#     return np.hstack([q_dot, p_dot, u_dot])

In [7]:
def propagate_dynamics(qpu_vec, sliding_window_instance):
    '''
    n_s is number of steps
    '''
    qs=[]
    ps=[]
    us=[]
    t_0, T, K, integrateTol, integrateMaxIter, state_dim, Gamma = sliding_window_instance.t_0, sliding_window_instance.T, sliding_window_instance.K, sliding_window_instance.integrateTol, sliding_window_instance.integrateMaxIter, sliding_window_instance.state_dim, sliding_window_instance.Gamma 
    #     ts = range(t_0,T+1,(T-t_0)/float(2*K))  # go until T+1 because last value will be used as starting point for next window
    ts = np.linspace(t_0, T, (2*K)+1)
    for i in range(len(ts)-1):
        # starting value of u for a single bucket
        t_start, t_end = ts[i], ts[i+1]
        q_0 = qpu_vec[:state_dim]
        p_0 = qpu_vec[state_dim:2*state_dim]
        u_0 = qpu_vec[2*state_dim:]
        qp_vecs = propagate_q_p(q_0, p_0, u_0, t_start, t_end, sliding_window_instance)  # assume "u" constant, and propagate q and p
        u_vecs = propagate_u(u_0, qp_vecs, t_start, t_end, sliding_window_instance)      # pass in the resulting q and p values to be used for propagating the "u"
        
        qpu_vec_i = np.hstack([qp_vecs, u_vecs])
        qpu_vec = qpu_vec_i[-1] # only need the last value
        if i == len(ts)-2:
            pass
            # no need to append since weight = 0 for last value.  But qpu_vec still needs to be updated.
        else:
            qs.append(qpu_vec[:state_dim])
            ps.append(qpu_vec[state_dim:2*state_dim])
            us.append(qpu_vec[2*state_dim:])
    return qpu_vec, qs, ps, us  # return values for one entire window

    
def propagate_q_p(q_0, p_0, u_0, t_start, t_end, sliding_window_instance):
    '''
    Propagate q and p to end of bucket using rk23
    '''
    n_s = sliding_window_instance.n_s
    qp_vecs = []
    qp_vec = np.concatenate([q_0, p_0])  # pass in all three: q_0, p_0, u_0, but in the qp_rhs function
    steps = np.linspace(t_start,t_end, n_s+1)
    for i in range(n_s):
        n_start, n_end = steps[i], steps[i+1]
        qp_vec, t, failFlag, iter_i = ode.ode_rk23(sliding_window_instance.qp_rhs, n_start, n_end, qp_vec, sliding_window_instance.integrateTol, sliding_window_instance.integrateMaxIter, state_dim=sliding_window_instance.state_dim, Gamma = sliding_window_instance.Gamma, u_0 = u_0)
        qp_vecs.append(qp_vec[-1])
        qp_vec = qp_vec[-1]
    return qp_vecs
    
    
def propagate_u(u_0, qp_vecs, t_start, t_end, sliding_window_instance):
    '''
    Propagate u based on q and p values
    u_vecs (list of 1-D numpy arrays):
    '''
    u_vecs = []
    u_vec = u_0
    n_s = sliding_window_instance.n_s
    steps = np.linspace(t_start,t_end, n_s+1)
    for i in range(n_s):
        n_start, n_end = steps[i], steps[i+1]
        qp_vec = qp_vecs[i]
        u_vec, t, failFlag, iter_i = ode.ode_rk23(sliding_window_instance.u_rhs, n_start, n_end, u_vec, sliding_window_instance.integrateTol, sliding_window_instance.integrateMaxIter, state_dim=sliding_window_instance.state_dim, Gamma = sliding_window_instance.Gamma, qp_vec = qp_vec)
        u_vecs.append(u_vec[-1]) # one u_vec for each step, append them and you have all the u_vecs for one bucket
        u_vec = u_vec[-1]
    return u_vecs
        

In [8]:
# print stop

In [9]:
mySlidingWindow.state_dim

1

#### Test the propagation

In [10]:
sliding_window_instance = mySlidingWindow

In [11]:
# print q_0
# print p_0
# print u_0
t_start = 0
t_end = 1
# print sliding_window_instance

qp_vecs =  propagate_q_p(sliding_window_instance.q_0, sliding_window_instance.p_0, sliding_window_instance.u_0, t_start, t_end, sliding_window_instance)


In [12]:
print len(qp_vecs)

10


In [13]:
# print stop

In [14]:
# print u_0
# print qp_vecs
# print t_start
# print t_end
# print n_s
# print sliding_window_instance
u_vecs = propagate_u(sliding_window_instance.u_0, qp_vecs, t_start, t_end, sliding_window_instance)

In [15]:
print len(u_vecs)

10


In [16]:
qpu_vec, qs, ps, us = propagate_dynamics(sliding_window_instance.qpu_vec, sliding_window_instance)

In [17]:
qpu_vec

array([0., 0., 0.])

In [18]:
qs

[array([0.])]

In [19]:
ps

[array([0.])]

In [20]:
us

[array([0.])]

### Triangle window filter

In [21]:
def get_weights(K):
    weights_0 = [float(i)/K for i in range(1,K+1)]  
    weights_1 = [2-(float(i)/K) for i in range(K+1,(2*K)+1)]
    # sanity check 
    assert len(weights_0)==len(weights_1)
    weights = weights_0+weights_1
    weights_total = sum(weights[:-1])
    return weights, weights_total

In [22]:
def apply_filter(vec, weights, weights_total):
    vec_weighted = [val*w for val,w in zip(vec, weights[:-1])]
    vec_current = np.sum(vec_weighted,0)
    vec_normalized = vec_current/float(weights_total)
    return vec_normalized

weights, weights_total = get_weights(sliding_window_instance.K)
q_bar = apply_filter(qs,weights, weights_total)
p_bar = apply_filter(ps,weights, weights_total)
u_bar = apply_filter(us,weights, weights_total)

In [23]:
sliding_window_instance.K

1

In [24]:
# outputs:

print q_bar
print p_bar
print u_bar

print qs
print ps
print us

[0.]
[0.]
[0.]
[array([0.])]
[array([0.])]
[array([0.])]


### Sliding window (outer loop)

In [25]:
# additional inputs 
t_terminal = 100

In [26]:
def sliding_window(sliding_window_instance):
    ''' 
    Inputs:
        t_0 (int): Initial time to start propagating dynamics
        T (int): End time of propagating dynamics 
        q_0 (np.array): initial values of state vector
        p_0 (np.array): initial values of costate vector
        u_0 (np.array): initial values of control vector
        state_dim (int): number of states
        Gamma (float): algorithmic parameter for Riemann descent algorithm
        t_terminal (int): time marking termination of control law propagator algorithm
    Outputs:
        q_bars, p_bars, u_bars (list of np.arrays): implemented state/costate/control values for entire propagator.
    ''' 
    t_0, T, K, q_0, p_0, u_0, state_dim, Gamma, t_terminal = sliding_window_instance.t_0, sliding_window_instance.T, sliding_window_instance.K, sliding_window_instance.q_0, sliding_window_instance.p_0, sliding_window_instance.u_0, sliding_window_instance.state_dim, sliding_window_instance.Gamma, sliding_window_instance.t_terminal
    q_bars = []
    p_bars = []
    u_bars = []
    weights, weights_total = get_weights(K)
    t=t_0 # wall clock time
    qpu_vec = np.hstack([q_0, p_0, u_0])
    while t<t_terminal:
            
        qpu_vec, qs, ps, us = propagate_dynamics(qpu_vec, sliding_window_instance)
        # qs, ps, and us will go to Mean Field somehow

        q_bar = apply_filter(qs,weights, weights_total)
        p_bar = apply_filter(ps,weights, weights_total)
        u_bar = apply_filter(us,weights, weights_total)
            
        t+=1

        q_bars.append(q_bar)
        p_bars.append(p_bar)
        u_bars.append(u_bar)

    return q_bars, p_bars, u_bars


In [27]:
q_bars, p_bars, u_bars = sliding_window(sliding_window_instance)

In [28]:
q_bars

[array([0.]), array([0.])]

In [29]:
p_bars

[array([0.]), array([0.])]

In [30]:
u_bars

[array([0.]), array([0.])]